In [ ]:
import pandas as pd
import numpy as np
from train3 import CompoundDatase
from torch.utils.data import DataLoader, SubsetRandomSampler, WeightedRandomSampler
from sklearn.model_selection import train_test_split

In [ ]:
data_dir = 'DATA2'
X_filename = 'X_pretrain'
y_filename = 'y_pretrain'
eval_name = 'val'

X_data_eval = pd.read_csv(f'{data_dir}/{X_filename}_{eval_name}.csv', index_col=0)
y_data_eval = pd.read_csv(f'{data_dir}/{y_filename}_{eval_name}.csv', index_col=0)

In [ ]:
def create_dataloaders(torch_dataset, batch_size, holdout_frac=0, shuffle=True, set_name='train', stratify=None):
    if holdout_frac == 0:
        return {set_name: DataLoader(torch_dataset, batch_size=batch_size, shuffle=shuffle)}

    # Get the targets of your dataset if it's available
    if stratify is not None:
        targets = [item[stratify] for item in torch_dataset]
    else:
        targets = None

    # Split the indices of your dataset
    indices = list(range(len(torch_dataset)))
    train_indices, holdout_indices = train_test_split(indices, test_size=holdout_frac, stratify=targets)

    # Create samplers
    train_sampler = SubsetRandomSampler(train_indices)
    holdout_sampler = SubsetRandomSampler(holdout_indices)

    return {set_name: DataLoader(torch_dataset, batch_size=batch_size, sampler=train_sampler),
            f'{set_name}_holdout': DataLoader(torch_dataset, batch_size=batch_size, sampler=holdout_sampler)}

In [ ]:
eval_dataset = CompoundDataset(X_data_eval,y_data_eval[y_head_cols], y_data_eval[y_adv_cols])

train_loader_dct = create_dataloaders(train_dataset, batch_size, holdout_frac, set_name=train_name)
eval_loader_dct = create_dataloaders(eval_dataset, batch_size, set_name = eval_name)